In [ ]:
pip install git+https://github.com/yuce/pyswip@master#egg=pyswip

In [ ]:
!head kb.txt

In [1]:
#TODO: use re
import re

triples = []
rels = set()

with open('data/kb.txt', 'r') as f:
    kb_data = f.read().strip().split('\n')
    for line in kb_data:
        elems = line.split('|')
        triples.append(elems)
        rels.add(elems[1])
        
print(len(triples))
triples[:5], rels


134741


([['Kismet', 'directed_by', 'William Dieterle'],
  ['Kismet', 'written_by', 'Edward Knoblock'],
  ['Kismet', 'starred_actors', 'Marlene Dietrich'],
  ['Kismet', 'starred_actors', 'Edward Arnold'],
  ['Kismet', 'starred_actors', 'Ronald Colman']],
 {'directed_by',
  'has_genre',
  'has_imdb_rating',
  'has_imdb_votes',
  'has_tags',
  'in_language',
  'release_year',
  'starred_actors',
  'written_by'})

In [94]:
def search_triples(s):
    for triple in triples:
        if s in triple[0] or s in triple[1] or s in triple[2]:
            print(triple)
            
search_triples("Frankenstein")

['The Evil of Frankenstein', 'directed_by', 'Freddie Francis']
['The Evil of Frankenstein', 'starred_actors', 'Peter Cushing']
['The Evil of Frankenstein', 'release_year', '1964']
['The Evil of Frankenstein', 'has_tags', 'peter cushing']
['The Ghost of Frankenstein', 'starred_actors', 'Ralph Bellamy']
['The Ghost of Frankenstein', 'starred_actors', 'Lon Chaney Jr.']
['The Ghost of Frankenstein', 'starred_actors', 'Cedric Hardwicke']
['The Ghost of Frankenstein', 'starred_actors', 'Lionel Atwill']
['The Ghost of Frankenstein', 'release_year', '1942']
['The Ghost of Frankenstein', 'has_genre', 'Horror']
["Jesse James Meets Frankenstein's Daughter", 'directed_by', 'William Beaudine']
["Jesse James Meets Frankenstein's Daughter", 'release_year', '1966']
["Jesse James Meets Frankenstein's Daughter", 'has_genre', 'Horror']
["Jesse James Meets Frankenstein's Daughter", 'has_genre', 'Western']
['House of Frankenstein', 'directed_by', 'Erle C. Kenton']
['House of Frankenstein', 'written_by', 'C

In [3]:
vocab = {}
for triple in triples:
    for i, ele in enumerate(triple):
        if ele not in vocab:
            if i == 0 or i == 2:
                vocab[ele] = f"ent_{len(vocab)}"
            else:
                vocab[ele] = f"rel_{len(vocab)}"


In [4]:
# add reverse relations

rev_rels = {f"{r}_reverse" for r in rels}
print(rev_rels)

for rel in rev_rels:
    vocab[rel] = f"rel_{len(vocab)}"
    

{'has_tags_reverse', 'in_language_reverse', 'has_genre_reverse', 'written_by_reverse', 'starred_actors_reverse', 'has_imdb_rating_reverse', 'release_year_reverse', 'has_imdb_votes_reverse', 'directed_by_reverse'}


In [5]:
from pyswip import Prolog
from tqdm import tqdm

prolog = Prolog()

for triple in tqdm(triples):
    s1, r, s2 = vocab[triple[0]], vocab[triple[1]], vocab[triple[2]]
    rev_r = vocab[f"{triple[1]}_reverse"]
    prolog.assertz(f"{r}({s1},{s2})")
    prolog.assertz(f"{rev_r}({s2},{s1})")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134741/134741 [00:04<00:00, 28858.59it/s]


## 1 hop

In [1]:
inv_vocab = {v:k for k, v in vocab.items()}

def query_obj(rel, s1):
    res = list(prolog.query(f"{vocab[rel]}({vocab[s1]},X)"))
    for d in res:
        d['X'] = inv_vocab[d['X']]
    return res

query_obj('written_by', 'Funny About Love')

NameError: name 'vocab' is not defined

## 2 hop

In [ ]:
f"{vocab['directed_by']}(X,{vocab['Yuriy Norshteyn']}), {vocab['written_by']}(X,Y)"

In [34]:
res = list(prolog.query(f"{vocab['directed_by']}(X,{vocab['Yuriy Norshteyn']}), {vocab['written_by']}(X,Y)"))
print(res)

for d in res:
    for k,v in d.items():
        d[k] = inv_vocab[v]
res

[{'X': 'ent_13004', 'Y': 'ent_13005'}, {'X': 'ent_13004', 'Y': 'ent_13005'}, {'X': 'ent_41313', 'Y': 'ent_41314'}, {'X': 'ent_41313', 'Y': 'ent_41314'}, {'X': 'ent_13004', 'Y': 'ent_13005'}, {'X': 'ent_13004', 'Y': 'ent_13005'}, {'X': 'ent_41313', 'Y': 'ent_41314'}, {'X': 'ent_41313', 'Y': 'ent_41314'}]


NameError: name 'inv_vocab' is not defined

## dataset to create

How many is enough? to answer I need to read the paper on meta learning

### 1-hop

what films did [Stewart Hopewell] direct -> directed_by(X,[Stewart Hopewell])
who was the director of [Fat Man and Little Boy] -> directed_by([Fat Man and Little Boy],X)
what was the release date of the movie [The Savages] -> release_year([The Savages],X)


### 2-hop


### 3-hop


## weak translation with manual labeling functions

In [ ]:
!wc -l ./metaqa/qa_train_3hop.txt
!wc -l ./metaqa/3hop-qtype.txt

In [ ]:
with open("./metaqa/qa_train_3hop.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    questions = []
    for line in lines:
        q, ans = line.split("\t")
        questions.append((q,ans))

with open("./metaqa/3hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    logics = []
    for line in lines:
        logics.append(line)


list(zip(questions, logics))[:5]

### extract all logical predicates from data

In [1]:

def logics_str_to_steps(logics_str):
    logics_str = logics_str.replace("_to_", "_")
    logics_str += "_"
    steps = []
    logic_steps = []
    i = 0
    prev_und_loc = -1
    while i < len(logics_str):
        if logics_str[i] == '_' or i == len(logics_str)-1:
            steps.append(logics_str[prev_und_loc+1:i])
            prev_und_loc = i
        i+=1
    for i in range(len(steps)-1):
        logic_steps.append(f"{steps[i]}_{steps[i+1]}")
    return logic_steps

logics_str_to_steps('movie_to_actor_to_movie_to_year')

['movie_actor', 'actor_movie', 'movie_year']

In [ ]:
logics_strs = []

with open("./metaqa/3hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        logics_strs.append(line)
        
with open("./metaqa/2hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        logics_strs.append(line)
        
with open("./metaqa/1hop-qtype.txt", 'r') as f:
    lines = f.read().strip().split('\n')
    for line in lines:
        logics_strs.append(line)
        
        
logics_strs_set = set()

for logic_str in logics_strs:
    steps = logics_str_to_steps(logic_str)
    logics_strs_set.update(steps)

logics_strs_set

### logic step to predicate

In [ ]:
logic_to_predicate_dict = {
    'actor_movie': 'starred_actors_reverse',
    'director_movie': 'directed_by_reverse',
    'movie_actor': 'starred_actors',
    'movie_director': 'directed_by',
    'movie_genre': 'has_genre',
    'movie_imdbrating': 'has_imdb_rating',
    'movie_imdbvotes': 'has_imdb_votes',
    'movie_language': 'in_language',
    'movie_tags': 'has_tags',
    'movie_writer': 'written_by',
    'movie_year': 'release_year',
    'tag_movie': 'has_tags_reverse',
    'writer_movie': 'written_by_reverse'
}

In [ ]:
import re

def create_query_string(question_str, question_steps, logic_to_predicate_dict, entity_rel_vocab):
    # extract question concept
    question_concept = re.findall(r'\[(.+)\]', question_str)[0]
    #print(question_concept)
    
    # define prolog variables
    prolog_vars = ['X', 'Y', 'Z']
    logic_steps = logics_str_to_steps(question_steps)
    predicate_steps = [logic_to_predicate_dict[x] for x in logic_steps]
    #print(logic_steps)
    #print(predicate_steps)
    
    # create predicates
    p1 = f'{predicate_steps[0]}({question_concept},X)'
    p1_query = f'{entity_rel_vocab[predicate_steps[0]]}({entity_rel_vocab[question_concept]},X)'
    i = 1
    predicates = [p1,]
    predicates_query = [p1_query,]
    while i < len(predicate_steps):
        predicates.append(f"{predicate_steps[i]}({prolog_vars[i-1]},{prolog_vars[i]})")
        predicates_query.append(
            f'{entity_rel_vocab[predicate_steps[i]]}({prolog_vars[i-1]},{prolog_vars[i]})'
        )
        i += 1
        
    query_string = ', '.join(predicates_query)
    return predicates, query_string
    
predicates, query = create_query_string(
    'what types are the films directed by the director of [For Love or Money]',
    'movie_to_director_to_movie_to_genre',
    logic_to_predicate_dict,
    vocab
)

In [ ]:
def interpret_query_result(query_result, eid_to_entity_dict):
    interpreted_res = []
    for result in query_result:
        d = {}
        for key, val in result.items():
            d[key] = eid_to_entity_dict[val]
        interpreted_res.append(d)
    return interpreted_res

print(predicates, query)
res=list(prolog.query(query))
interpret_query_result(res, inv_vocab)

### build the dataset

In [ ]:
questions = []
logics = []

for i in range(1,4):
    with open(f"./metaqa/qa_train_{i}hop.txt", 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines:
            q, ans = line.split("\t")
            questions.append((q,ans))

    with open(f"./metaqa/{i}hop-qtype.txt", 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines:
            logics.append(line)
        


list(zip(questions, logics))[:5]

In [ ]:
from sklearn.model_selection import train_test_split

raw_dataset = list(zip(questions, logics))
train_ds, test_ds = train_test_split(raw_dataset, test_size=0.05)
len(train_ds), len(test_ds)

In [ ]:
train_ds[:5]

In [ ]:
import pandas as pd

def build_csv_dataset(ds, path):
    translated = []
    questions = []
    for (question, ans), logic in ds:
        questions.append(question)
        predicates, query = create_query_string(
            question,
            logic,
            logic_to_predicate_dict,
            vocab
        )
        translated.append(",".join(predicates))
    df = pd.DataFrame({'question': questions, 'logic': translated})
    #df.to_csv(path, index=False, header=True)
    df.to_json(path, orient='records', lines=True)

build_csv_dataset(train_ds, 'train_s2s.jsonl')
build_csv_dataset(test_ds, 'valid_s2s.jsonl')

In [ ]:
!head train_s2s.json

## test model

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./results/t5-v1/checkpoint-4000/")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("./results/t5-v1/checkpoint-4000/")

In [ ]:
sent = "summarize: the films directed by the director of [Heaven Is for Real] starred who"
#,"logic":"directed_by(Heaven Is for Real,X),directed_by_reverse(X,Y),starred_actors(Y,Z)""
inputs = tokenizer(
    f"summarize: {sent}",
    return_tensors="pt",
#     max_length=100,
#     truncation=True
).input_ids
outputs = model.generate(inputs, max_new_tokens=200)#, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
model.config

In [ ]:
inputs = tokenizer(
    [f"summarize: {sent}", "which mother fuckers played in movie [navid in wonderland]"],
    return_tensors="pt",
    max_length=100,
    truncation=True,
    padding=True
).input_ids

outputs = model.generate(inputs, max_length=50)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

### accuracy of translator on test set

In [29]:
import json

questions = []
trans_qs = []
with open('data/test.json', 'r') as json_file:
    json_list = list(json_file)
    for json_str in json_list:
        result = json.loads(json_str)
        questions.append(result['question'])
        trans_qs.append(result['logical_steps'])

# preprocess questions
questions = [f"summarize: {q}" for q in questions]
questions[:3], len(questions)

(['summarize: what does [Grégoire Colin] appear in',
  'summarize: [Joe Thomas] appears in which movies',
  'summarize: what films did [Michelle Trachtenberg] star in'],
 39093)

In [ ]:
from tqdm import tqdm

def get_translated_output(query_list, tokenizer, model, batch_size=5000):
    results = []
    i = 0
    pbar = tqdm(total = len(query_list))
    
    while i < len(query_list):
        batch = query_list[i:i+batch_size]
        
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            max_length=200,
            truncation=True,
            padding=True
        ).input_ids
        
        outputs = model.generate(inputs, max_length=200)
        
        curr_res = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]
        i += batch_size
        pbar.update(batch_size)
        results.extend(curr_res)
    pbar.close()
    return results
    
model_outputs = get_translated_output(questions, tokenizer, model)

In [ ]:
exact_matches = 0

for model_gen, gold_out in zip(model_outputs, trans_qs):
    if model_gen == gold_out:
        exact_matches += 1
    else:
        print(model_gen, gold_out)
        
print(f"accuracy: {exact_matches/len(model_outputs)}")

In [ ]:
import pickle

with open('predictions.pkl', 'wb') as f:
    pickle.dump(model_outputs, f)

In [ ]:
len(model_outputs), len(trans_qs)

## accuracy of qa model

In [6]:
import pickle 

with open('data/predictions.pkl', 'rb') as f:
    model_outputs = pickle.load(f)

In [7]:
model_outputs[20000:20004]

['written_by_reverse(F. Paul Benz,X), starred_actors(X,Y)',
 'written_by_reverse(Peter M. Lenkov,X), release_year(X,Y)',
 'directed_by(Salmon Fishing in the Yemen,X), directed_by_reverse(X,Y)',
 'written_by_reverse(Mark Crowdy,X), starred_actors(X,Y)']

In [43]:
rels = {}
for k, v in vocab.items():
    if 'rel_' in v:
        rels[k] = v
rels

{'directed_by': 'rel_1',
 'written_by': 'rel_3',
 'starred_actors': 'rel_5',
 'release_year': 'rel_10',
 'in_language': 'rel_12',
 'has_tags': 'rel_14',
 'has_genre': 'rel_22',
 'has_imdb_votes': 'rel_24',
 'has_imdb_rating': 'rel_216',
 'has_tags_reverse': 'rel_43243',
 'in_language_reverse': 'rel_43244',
 'has_genre_reverse': 'rel_43245',
 'written_by_reverse': 'rel_43246',
 'starred_actors_reverse': 'rel_43247',
 'has_imdb_rating_reverse': 'rel_43248',
 'release_year_reverse': 'rel_43249',
 'has_imdb_votes_reverse': 'rel_43250',
 'directed_by_reverse': 'rel_43251'}

In [44]:
ents = {k:v for k,v in vocab.items() if k not in rels}
print(len(ents), len(vocab))

43234 43252


In [54]:
from tqdm import tqdm
import re
from rapidfuzz.distance import Levenshtein

def find_entity_by_ed(q_ent):
    for ent in vocab:
        d = Levenshtein.distance(q_ent, ent, score_cutoff=2)
        if d < 3:
            return ent
    return None
    

def transform_query_to_prolog_form(query, question_ent):
    rels_sorted_by_len = sorted(list(rels.keys()), key=lambda x: -len(x))
    for rel in rels_sorted_by_len:
        if rel in query:
            query = query.replace(rel, rels[rel])
    
    try:
        query = query.replace(question_ent, ents[question_ent])
    except KeyError:
        question_ent_rec = find_entity_by_ed(question_ent)
        if question_ent_rec is None:
            raise Exception("entity not found!")
        else:
#             print(f"RECOVERED {question_ent_rec}")
#             print(f"replacing {question_ent_rec} with {ents[question_ent_rec]}")
            
            query = query.replace(question_ent, ents[question_ent_rec])
#             print(f"results: {query}")
        
    return query
    
transform_query_to_prolog_form(
    'directed_by(Salmon Fishing in the Yemen,X), directed_by_reverse(X,Y)',
    'Salmon Fishing in the Yemen'
)

'rel_1(ent_13436,X), rel_43251(X,Y)'

In [92]:
inv_vocab = {v:k for k,v in vocab.items()}
def find_all_answers(query):

    question_ent = re.findall(r'\((.+),X\)', query)[0]
    prolog_query = transform_query_to_prolog_form(query, question_ent)
    results = list(prolog.query(prolog_query))
    print(results)
    answers = []
    for res in results:
        for k, v in res.items():
            res[k] = inv_vocab[v]
        if len(res) == 1:
            answers.append(res['X'])
        elif len(res) == 2:
            answers.append(res['Y'])
        elif len(res) == 3:
            answers.append(res['Z'])
    return [x for x in list(set(answers)) if x != question_ent]
            
    
find_all_answers('release_year(Frankenstein,X)')

[{'X': 'ent_21828'}, {'X': 'ent_1605'}]


['1931', '1910']

In [78]:
predicted_answers = []

for q, out in tqdm(zip(questions, model_outputs)):
    try:
        ans_set = find_all_answers(out)
        predicted_answers.append(ans_set)
    except Exception as e:
        predicted_answers.append([])
        print("*"*100)
        print(e)
        print(f"FAILED FOR: {q}")
        
    
    

4015it [00:00, 7784.58it/s]

****************************************************************************************************
Caused by: 'rel_ent_3821(ent_3821,X)'. Returned: 'error(existence_error(procedure, /(rel_ent_3821, 2)), context(/(pyrun, 2), _174))'.
FAILED FOR: summarize: the director of [1] was
****************************************************************************************************
entity not found!
FAILED FOR: summarize: who was [The Way Way Back] directed by


7967it [00:01, 9118.80it/s]

****************************************************************************************************
entity not found!
FAILED FOR: summarize: who in the world wrote [Faster, Pussycat! Kill! Kill!]


9807it [00:01, 7510.82it/s]

...

34370it [00:31, 375.65it/s]

****************************************************************************************************
entity not found!
FAILED FOR: summarize: the movies directed by the director of [Tora! Tora! Tora!] starred who


34901it [00:32, 461.85it/s]

****************************************************************************************************
entity not found!
FAILED FOR: summarize: what genres are the movies written by [Faster, Pussycat! Kill! Kill!] writers


39093it [00:42, 921.69it/s]


In [79]:
predicted_answers[:5]

[['Before the Rain'],
 ['The Inbetweeners Movie', 'The Inbetweeners 2'],
 ['The Scribbler',
  'Harriet the Spy',
  'Black Christmas',
  'Inspector Gadget',
  'Ice Princess'],
 ['Kiss and Make-Up', 'The Son of Kong', 'Divorce'],
 ['Haider', 'Chance Pe Dance', 'Jab We Met']]

In [80]:
import os

hops = ['1hop', '2hop', '3hop']

answers = {}

for hop in hops:
    answers[hop] = []
    path = os.path.join('data', hop, 'qa_test.txt')
    with open(path, 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines:
            _, a = line.split('\t')
            answers[hop].append(a.strip().split('|'))


In [105]:
# 1-hop accuracy (all or nothing)
cnt = 0
for ans, pred ,q, model_out in zip(answers['1hop'], predicted_answers[:len(answers['1hop'])],questions[:len(answers['1hop'])], model_outputs[:len(answers['1hop'])]):
    if set(pred).issubset(set(ans)):
        cnt += 1
    else:
        pass
        print("*"*100)
        print(ans, pred, q, model_out)
        
print(cnt/len(answers['1hop']))

****************************************************************************************************
['All About Eve', 'Sleuth', 'Cleopatra', 'Guys and Dolls', 'Suddenly', 'Last Summer', 'Julius Caesar', 'The Barefoot Contessa', 'A Letter to Three Wives', 'People Will Talk', 'No Way Out', '5 Fingers', 'There Was a Crooked Man...', 'Dragonwyck', 'House of Strangers', 'Somewhere in the Night', 'The Honey Pot', 'The Quiet American', 'A Carol for Another Christmas'] ['5 Fingers', 'People Will Talk', 'Guys and Dolls', 'There Was a Crooked Man...', 'The Barefoot Contessa', 'The Honey Pot', 'Julius Caesar', 'A Carol for Another Christmas', 'Suddenly, Last Summer', 'Cleopatra', 'Sleuth', 'Somewhere in the Night', 'A Letter to Three Wives', 'The Quiet American', 'No Way Out', 'House of Strangers', 'Dragonwyck', 'All About Eve'] summarize: [Joseph L. Mankiewicz] directed which movies directed_by_reverse(Joseph L. Mankiewicz,X)
*********************************************************************

In [77]:
len(predicted_answers), len(model_outputs)

(39105, 39093)

In [97]:
# 3-hop accuracy (all or nothing)
start = len(answers['1hop']) + len(answers['2hop'])

cnt = 0
for ans, pred ,q, model_out in zip(answers['3hop'], predicted_answers[start:], questions[start:], model_outputs[start:]):
    if set(ans).issubset(set(pred)):
        cnt += 1
        if set(ans) != set(pred):
            print(ans, pred, q, model_out)
    else:
        pass
#         print("*"*100)
#         print(ans, pred, q, model_out)
        
print(cnt/len(answers['3hop']))

['Sandra Dee', 'Charles Coburn', 'Cornel Wilde', 'John Gavin', 'Warren William', 'Susan Kohner', 'Joan Bennett', 'Fred MacMurray', 'Barbara Stanwyck', 'Don Ameche', 'Jane Wyman', 'Rochelle Hudson', 'Boris Karloff', 'Patricia Knight', 'Robert Cummings', 'Rock Hudson', 'Lucille Ball', 'Claudette Colbert', 'Lana Turner', 'George Sanders'] ['Lucille Ball', 'Lauren Bacall', 'Warren William', 'Fred MacMurray', 'John Gavin', 'Barbara Stanwyck', 'Susan Kohner', 'Dorothy Malone', 'Rock Hudson', 'Boris Karloff', 'Don Ameche', 'Sandra Dee', 'Patricia Knight', 'Jane Wyman', 'Joan Bennett', 'Robert Cummings', 'Cornel Wilde', 'Lana Turner', 'Claudette Colbert', 'Rochelle Hudson', 'Robert Stack', 'Charles Coburn', 'George Sanders'] summarize: who starred movies for the director of [Written on the Wind] directed_by(Written on the Wind,X), directed_by_reverse(X,Y), starred_actors(Y,Z)
['1995', '1996', '1999', '1998', '1989', '2002', '2000', '2008', '2011', '2010'] ['1989', '1995', '1999', '2008', '2013

['Barry W. Blaustein', 'Walter Hill', 'Mario Bava', 'Daniel Sackheim', 'Baz Luhrmann', 'Jack Nicholson', 'John Frankenheimer', 'Bob Brooks', 'Michael Ritchie', 'Mark Rydell', 'Mary Stuart Masterson', 'Joe Dante', 'Alfred Hitchcock', 'Elliott Nugent', 'Jeffrey Kramer', 'Jack Clayton', 'Jason Miller', 'Hal Ashby', 'Alexander Payne', 'Roger Corman', 'James Foley', 'Bob Rafelson'] ['Daniel Sackheim', 'Alexander Payne', 'Jack Nicholson', 'John Frankenheimer', 'Roger Corman', 'Alfred Hitchcock', 'James Foley', 'Mary Stuart Masterson', 'Elliott Nugent', 'Jeffrey Kramer', 'Barry W. Blaustein', 'Bob Rafelson', 'Douglas Trumbull', 'Joe Dante', 'Baz Luhrmann', 'Bob Brooks', 'Jason Miller', 'Jack Clayton', 'Michael Ritchie', 'Walter Hill', 'Hal Ashby', 'Mario Bava', 'Mark Rydell'] summarize: who directed films that share actors with the film [Silent Running] starred_actors(Silent Running,X), starred_actors_reverse(X,Y), directed_by(Y,Z)
['1987', '1984', '1980', '1971', '1979'] ['1989', '1979', '19

['Kazuo Ishiguro', 'Alex Garland'] ['Mark Romanek', 'Alex Garland', 'Kazuo Ishiguro'] summarize: who are the screenwriters that the directors of their movies also directed [One Hour Photo] directed_by(One Hour Photo,X), directed_by_reverse(X,Y), written_by(Y,Z)
['Jay Lacopo', 'Jordan Brady'] ['Jordan Brady', 'Mark Fauser', 'Brent Briscoe', 'Jay Lacopo'] summarize: the movies that share directors with the movie [Waking Up in Reno] are written by who directed_by(Waking Up in Reno,X), directed_by_reverse(X,Y), written_by(Y,Z)
['2002', '1993', '1985'] ['1989', '1985', '2002', '1993'] summarize: when did the movies directed by the [K-9] director release directed_by(K-9,X), directed_by_reverse(X,Y), release_year(Y,Z)
['English'] ['English', 'German'] summarize: what are the languages spoken in the movies written by [Rosalie Goes Shopping] writers written_by(Rosalie Goes Shopping,X), written_by_reverse(X,Y), in_language(Y,Z)
['Halle Berry', 'Craig Sheffer', 'Teri Garr', 'Charlie Sheen', 'Mac 

['Alexis Denisof', 'Mary Ure', 'Jackie Cooper', 'Elina Löwensohn', 'Warren Oates', 'Joely Richardson', 'Peter Fonda', 'Helena Bonham Carter', 'Jodie Foster', 'Orson Welles', 'Michael Caine', 'Tommy Lee Jones', 'Mark McManus', 'George C. Scott', 'Natalie Portman', 'Judi Bowker', 'Chris Hemsworth', 'Michael Redgrave', 'Plácido Domingo', 'Powers Boothe', 'Mary Pickford', 'Justino Díaz', 'Kenneth Branagh', 'Tom Courtenay', 'Valerie Perrine', 'Hugh Laurie', 'Ricky Schroder', 'Richard Madden', 'Clark Gregg', 'Jonathan Winters', 'Brooke Shields', 'Heath Ledger', 'Bryce Dallas Howard', 'Claire Bloom', 'Wayne Knight', 'Elizabeth Taylor', 'Alex Pettyfer', 'Alicia Silverstone', 'Leonard Whiting', 'Anthony Hopkins', 'Derek Jacobi', 'Ian McKellen', 'Faye Dunaway', 'Harold Pinter', 'Mick Jagger', 'Katia Ricciarelli', 'Norma Shearer', 'Robert Downey Jr.', 'Jessica Lange', 'Olivia Hussey', 'Anjanette Comer', 'Diana Ross', 'Jack Nicholson', 'Joan Fontaine', 'Cate Blanchett', 'Jim Broadbent', 'Cedric Ha

['James M. Cain', 'Wendell Mayes', 'David Rayfiel', 'Alexandre Breffort', 'I.A.L. Diamond', 'Ernest Lehman', 'Ferenc Molnár', 'Ben Hecht', 'Gilles Mimouni', 'Charles Brackett', 'Raymond Chandler', 'Harry Kurnitz', 'Barbara Benedek', 'Anna Bonacci', 'Robert Harari', 'Charles R. Jackson', 'Charles Lederer', 'Francis Veber', 'Claude Anet', 'Richard L. Breen', 'Agatha Christie', 'Edwin Blum', 'Bartlett Cormack', 'Edmund Trzcinski', 'Charles MacArthur', 'Billy Wilder', 'Samuel A. Taylor', 'David Shaw', 'Donald Bevan', 'George Axelrod'] ['Raymond Chandler', 'Harry Kurnitz', 'Ben Hecht', 'Ernest Lehman', 'Robert Harari', 'Edmund Trzcinski', 'George Axelrod', 'Bartlett Cormack', 'Charles R. Jackson', 'Edwin Blum', 'David Shaw', 'Donald Bevan', 'Charles Lederer', 'Wendell Mayes', 'James M. Cain', 'I.A.L. Diamond', 'Alexandre Breffort', 'Billy Wilder', 'Samuel A. Taylor', 'Ferenc Molnár', 'Charles MacArthur', 'Francis Veber', 'Agatha Christie', 'Edward Childs Carpenter', 'Claude Anet', 'Barbara 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['1986', '1990', '1993', '1995', '1997', '1987', '1988', '1991']
['1997', '1990', '1995', '1988', '1991', '1987', '1998', '1986', '1993']
summarize: the movies that share directors with the movie [Ernest in the Army] were released in which years directed_by(Ernest in the Army,X), directed_by_reverse(X,Y), release_year(Y,Z)

In [104]:
results = list(prolog.query(
    f"{vocab['directed_by']}({vocab['Ernest in the Army']},X), {vocab['directed_by_reverse']}(X,Y), "
    f"{vocab['release_year']}(Y,Z), not(Y=={vocab['Ernest in the Army']})"
))

for res in results:
    for k, v in res.items():
        res[k] = inv_vocab[v]
        
results

[{'X': 'John R. Cherry III', 'Y': 'Ernest Goes to Africa', 'Z': '1997'},
 {'X': 'John R. Cherry III', 'Y': 'Ernest Scared Stupid', 'Z': '1991'},
 {'X': 'John R. Cherry III', 'Y': 'Ernest Saves Christmas', 'Z': '1988'},
 {'X': 'John R. Cherry III',
  'Y': 'Dr. Otto and the Riddle of the Gloom Beam',
  'Z': '1986'},
 {'X': 'John R. Cherry III', 'Y': 'Slam Dunk Ernest', 'Z': '1995'},
 {'X': 'John R. Cherry III', 'Y': 'Ernest Goes to Jail', 'Z': '1990'},
 {'X': 'John R. Cherry III', 'Y': 'Ernest Goes to Camp', 'Z': '1987'},
 {'X': 'John R. Cherry III', 'Y': 'Ernest Rides Again', 'Z': '1993'}]

## analyze self loop questions

### 2 hops

In [46]:
questions = []
logic_steps = []

with open('data/2hop/qa_train.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        questions.append(line.strip().split('\t')[0])
        
with open('data/2hop/qa_train_qtype.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        steps = line.strip().split('_')
        steps = [s for s in steps if s!='to']
        logic_steps.append(steps)
        
other_qs = []
complex_logic_steps = []
filtered_questions = []
for q, steps in zip(questions, logic_steps):
    if steps[0] == steps[-1]:
        filtered_questions.append(q)
        complex_logic_steps.append(steps)
    else:
        other_qs.append(q)
        
list(zip(filtered_questions[:100], complex_logic_steps[:100])), len(filtered_questions), len(questions)

([('which movies have the same director of [Just Cause]',
   ['movie', 'director', 'movie']),
  ('what are the movies that have the same screenwriter of [Dodsworth]',
   ['movie', 'writer', 'movie']),
  ('the screenwriter [Mario Van Peebles] co-wrote movies with who',
   ['writer', 'movie', 'writer']),
  ('which movies share the same actor of [The Strangers]',
   ['movie', 'actor', 'movie']),
  ('which movies have the same screenwriter of [Cinderella]',
   ['movie', 'writer', 'movie']),
  ('the director of [Five Weeks in a Balloon] also directed which movies',
   ['movie', 'director', 'movie']),
  ('the actor of [So Undercover] also starred in which movies',
   ['movie', 'actor', 'movie']),
  ('which films have the same actor of [Echoes of the Rainbow]',
   ['movie', 'actor', 'movie']),
  ('which films share the screenwriter with [Double Suicide]',
   ['movie', 'writer', 'movie']),
  ('the director of [The Rookie] is also the director of which films',
   ['movie', 'director', 'movie'])

#### iterative label finding

In [65]:
filter_out_list = [
    'have the same',
    'share the same',
    'share the',
    'co-wrote',
    'also directed',
    'also starred',
    'also wrote',
    'acted together',
    'starred together',
    'co-writers',
    'co-star',
    'co-directed',
    # we can cut this list short and see if model generalize or not
    'also appears',
    'same director',
    'wrote movies together',
    'wrote films together',
    'co-directors',
    'appeared in the same',
    'together with',
]

new_qs = []
new_steps = []

for q, step in zip(filtered_questions, complex_logic_steps):
    for s in filter_out_list:
        if s in q:
            break
    else:
        new_qs.append(q)
        new_steps.append(step)

len(new_qs), list(zip(new_qs, new_steps))[:100]

(0, [])

#### check how many wrong labels we have

In [48]:
cnt = 0
for q in other_qs:
    for s in filter_out_list:
        if s in q:
            print(s, q)
            cnt += 1
            break
            
print(f'{cnt} out of {len(other_qs)}')

0 out of 78713


#### observed weak labels

if step 0 equals step -1:
- have the same X
- share the same Y
- co-wrote
- also directed
- also starred

### 3 hops

In [55]:
questions = []
logic_steps = []

with open('data/3hop/qa_train.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        questions.append(line.strip().split('\t')[0])
        
with open('data/3hop/qa_train_qtype.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        steps = line.strip().split('_')
        steps = [s for s in steps if s!='to']
        logic_steps.append(steps)
        
        
_0_2_qs = []
_1_3_qs = []
_0_2_steps = []
_1_3_steps = []
other_qs = []

for q, steps in zip(questions, logic_steps):
    if steps[0] == steps[2]:
        _0_2_qs.append(q)
        _0_2_steps.append(steps)
    elif sptes[1] == steps[3]:
        _1_3_steps.append(steps)
        _1_3_qs.append(q)
        
len(questions), len(_0_2_qs), len(_1_3_qs), list(zip(_0_2_qs, _0_2_steps))[:100]


(114196,
 114196,
 0,
 [('the films that share actors with the film [Dil Chahta Hai] were released in which years',
   ['movie', 'actor', 'movie', 'year']),
  ('who are the directors of the movies written by the writer of [The Green Mile]',
   ['movie', 'writer', 'movie', 'director']),
  ('which person directed the films acted by the actors in [Jawbreaker]',
   ['movie', 'actor', 'movie', 'director']),
  ('who is listed as director of the movies starred by [December Boys] actors',
   ['movie', 'actor', 'movie', 'director']),
  ('what types are the films directed by the director of [For Love or Money]',
   ['movie', 'director', 'movie', 'genre']),
  ('who wrote films that share actors with the film [Anastasia]',
   ['movie', 'actor', 'movie', 'writer']),
  ('the movies that share actors with the movie [The Constant Nymph] were released in which years',
   ['movie', 'actor', 'movie', 'year']),
  ('when did the movies release whose actors also appear in the movie [Cast a Deadly Spell]',
 

In [66]:
filter_out_list = [
    'share actors',
    'share directors',
    'share writers',
    'share actors',
    'also directed',
    'also appear',
    'also wrote',
    'share screenwriters'
]

new_qs = []
new_steps = []

for q, step in zip(_0_2_qs, _0_2_steps):
    for s in filter_out_list:
        if s in q:
            break
    else:
        new_qs.append(q)
        new_steps.append(step)

len(new_qs), len(_0_2_qs), list(zip(new_qs, new_steps))[-300:]

(54549,
 114196,
 [('when did the movies written by [The Kentucky Fried Movie] writers release',
   ['movie', 'writer', 'movie', 'year']),
  ('when did the movies directed by the [Super Sucker] director release',
   ['movie', 'director', 'movie', 'year']),
  ('which person wrote the films directed by the director of [Shampoo]',
   ['movie', 'director', 'movie', 'writer']),
  ('what were the release dates of films directed by the director of [Bianco, rosso e Verdone]',
   ['movie', 'director', 'movie', 'year']),
  ("which person directed the films acted by the actors in [Don't Look Back]",
   ['movie', 'actor', 'movie', 'director']),
  ('what genres are the films starred by [Burnt Offerings] actors',
   ['movie', 'actor', 'movie', 'genre']),
  ("who are the actors in the movies written by the writer of [Boys Don't Cry]",
   ['movie', 'writer', 'movie', 'actor']),
  ('the movies written by the screenwriter of [Next] were directed by who',
   ['movie', 'writer', 'movie', 'director']),
  (